# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

print('WORKSPACE DETAILS:', 'Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

experiment_name = 'Heart_Disease_Kaggle'
experiment=Experiment(workspace=ws, name=experiment_name)

run = experiment.start_logging()

WORKSPACE DETAILS:
Workspace name: quick-starts-ws-136845
Azure region: southcentralus
Subscription id: 610d6e37-4747-4a20-80eb-3aad70a55f43
Resource group: aml-quickstarts-136845


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# Creating compute cluster
# vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes = 4.

cpu_cluster_name = "cpu-cluster"

# to check whether the compute cluster exists already or not
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Existing compute target found... Using it")

except ComputeTargetException:
    print("Creating new Compute Target... " + cpu_cluster_name)
    provisioning_compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, provisioning_compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Existing compute target found... Using it
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Datset Preview

In [4]:
from azureml.data.dataset_factory import TabularDatasetFactory
import pandas as pd

url_path = "https://raw.githubusercontent.com/bharati-21/AZMLND-Capstone-Project/master/files/heart.csv"
# ds = TabularDatasetFactory.from_delimited_files(path=url_path)

df = pd.read_csv(url_path)

In [5]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [6]:
from azureml.widgets import RunDetails
import shutil
from azureml.train.sklearn import SKLearn
from azureml.core import ScriptRunConfig, Experiment
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import BayesianParameterSampling
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.train.hyperdrive import HyperDriveConfig
import os


# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
# early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1, delay_evaluation=5)

# TODO: Create the different params that you will be using during training
param_sampling = BayesianParameterSampling({
    "--C" : uniform(0.01, 1),
    "--max_iter" : choice(10, 50, 100, 150, 200),
})

#TODO: Create your estimator and hyperdrive config
if "training" not in os.listdir():
    os.mkdir("./training")

estimator = SKLearn (
    source_directory= os.path.join("./"),
    compute_target= cpu_cluster,
    entry_script= "train.py"
)

hyperdrive_run_config = HyperDriveConfig (
        estimator = estimator, 
        hyperparameter_sampling = param_sampling, 
        policy = None,
        primary_metric_name = 'accuracy', 
        primary_metric_goal = PrimaryMetricGoal.MAXIMIZE, 
        max_total_runs = 20,
        max_concurrent_runs = 4
)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
For best results with Bayesian Sampling we recommend using a maximum number of runs greater than or equal to 20 times the number of hyperparameters being tuned. Recommendend value:40.


In [7]:
#TODO: Submit your experiment

run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(run).show()
run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_58e6e3e4-a6dd-497f-a556-ad0df58da191
Web View: https://ml.azure.com/experiments/Heart_Disease_Kaggle/runs/HD_58e6e3e4-a6dd-497f-a556-ad0df58da191?wsid=/subscriptions/610d6e37-4747-4a20-80eb-3aad70a55f43/resourcegroups/aml-quickstarts-136845/workspaces/quick-starts-ws-136845

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-02-01T15:20:58.911542][API][INFO]Experiment created<END>\n""<START>[2021-02-01T15:21:00.490882][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"<START>[2021-02-01T15:21:00.2484621Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2021-02-01T15:21:01.074385][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"


## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [12]:
import joblib
# Get your best run and save the model from that run.

best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']
print('Best Run ID: ', best_run.id)
print('Best Run Accuracy: ', best_run_metrics['accuracy'])
print('Best Run Learning Rate: ', parameter_values[3])
print('Parameter Values: ', parameter_values)

Best Run ID:  HD_4c834d0c-e97e-4676-b0b4-8d5768b1e812_4
Best Run Accuracy:  0.9120879120879121
Best Run Learning Rate:  200
Parameter Values:  ['--C', '0.7302586180159412', '--max_iter', '200']


In [13]:
import joblib

#TODO: Save the best model

registered_model = best_run.register_model(model_name='heart_disease_hyperdrive_best_model', model_path='./outputs/model.joblib')
print("Model successfully saved.")

Model successfully saved.


In [14]:
best_run.get_file_names()

['azureml-logs/55_azureml-execution-tvmps_b631682e542fa6a24e2014116a3ecb02364d1891b686cac6b2d77e0889755d57_d.txt',
 'azureml-logs/65_job_prep-tvmps_b631682e542fa6a24e2014116a3ecb02364d1891b686cac6b2d77e0889755d57_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_b631682e542fa6a24e2014116a3ecb02364d1891b686cac6b2d77e0889755d57_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/103_azureml.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/model.joblib']

In [15]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
Heart_Disease_Kaggle,HD_4c834d0c-e97e-4676-b0b4-8d5768b1e812_4,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service